# Neural Networks - Simplified Version for easy experiments

## _Important_ if you let this run on Colab switch on the GPU option, because otherwise this part will take quite some time

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
%matplotlib inline
%pylab inline
import matplotlib.pyplot as plt

Populating the interactive namespace from numpy and matplotlib


In [3]:
import pandas as pd
print(pd.__version__)

0.23.4


In [4]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
print(tf.__version__)

1.10.0


In [5]:
# let's see what compute devices we have available, hopefully a GPU 
sess = tf.Session()
devices = sess.list_devices()
for d in devices:
    print(d.name)

/job:localhost/replica:0/task:0/device:CPU:0
/job:localhost/replica:0/task:0/device:GPU:0


In [6]:
# a small sane check, does tf seem to work ok?
hello = tf.constant('Hello TF!')
print(sess.run(hello))

b'Hello TF!'


## Loading and preparing our data set for classification

In [7]:
!curl -O https://raw.githubusercontent.com/DJCordhose/ai/master/notebooks/manning/data/insurance-customers-1500.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 26822  100 26822    0     0   139k      0 --:--:-- --:--:-- --:--:--  139k


In [8]:
df = pd.read_csv('./insurance-customers-1500.csv', sep=';')
y=df['group']
df.drop('group', axis='columns', inplace=True)
X = df.as_matrix()

In [9]:
df.head()

,max speed,age,thousand miles per year
0,98.0,44.0,25.0
1,118.0,54.0,24.0
2,111.0,26.0,34.0
3,97.0,25.0,10.0
4,114.0,38.0,22.0


In [10]:
df.describe()

,max speed,age,thousand miles per year
count,1500.000000,1500.000000,1500.000000
mean,122.492667,44.952667,30.344000
std,17.604333,17.191727,15.463152
min,68.000000,9.000000,-21.000000
25%,108.000000,32.000000,18.000000
50%,120.000000,42.000000,29.000000
75%,137.000000,55.000000,42.000000
max,166.000000,102.000000,84.000000


In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42, stratify=y)

In [13]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((900, 3), (900,), (600, 3), (600,))

In [14]:
dropout = 0.6
model = tf.keras.Sequential()

model.add(tf.keras.layers.Dense(100, name='hidden1', input_dim=3))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dropout(dropout))

model.add(tf.keras.layers.Dense(100, name='hidden2'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dropout(dropout))

model.add(tf.keras.layers.Dense(3, name='softmax', activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden1 (Dense)              (None, 100)               400       
_________________________________________________________________
batch_normalization (BatchNo (None, 100)               400       
_________________________________________________________________
activation (Activation)      (None, 100)               0         
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
hidden2 (Dense)              (None, 100)               10100     
_________________________________________________________________
batch_normalization_1 (Batch (None, 100)               400       
_________________________________________________________________
activation_1 (Activation)    (None, 100)               0         
__________

In [15]:
BATCH_SIZE=1000
EPOCHS = 2000

# only if you are running this locally
# !rm -rf ./tf_log
# %time model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=0.2, callbacks=[tb_callback])

%time history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=0.2)

Train on 720 samples, validate on 180 samples
Epoch 1/2000
720/720 [==============================] - 1s 1ms/step - loss: 1.7310 - acc: 0.3042 - val_loss: 6.1280 - val_acc: 0.3333
Epoch 2/2000
720/720 [==============================] - 0s 25us/step - loss: 1.6168 - acc: 0.3472 - val_loss: 6.1392 - val_acc: 0.3389
Epoch 3/2000
720/720 [==============================] - 0s 12us/step - loss: 1.6032 - acc: 0.3403 - val_loss: 6.1795 - val_acc: 0.3444
Epoch 4/2000
720/720 [==============================] - 0s 14us/step - loss: 1.6160 - acc: 0.3444 - val_loss: 6.3141 - val_acc: 0.3278
Epoch 5/2000
720/720 [==============================] - 0s 12us/step - loss: 1.4884 - acc: 0.3569 - val_loss: 6.4962 - val_acc: 0.3278
Epoch 6/2000
720/720 [==============================] - 0s 15us/step - loss: 1.4744 - acc: 0.3806 - val_loss: 6.7712 - val_acc: 0.3278
Epoch 7/2000
720/720 [==============================] - 0s 12us/step - loss: 1.4315 - acc: 0.3611 - val_loss: 7.0664 - val_acc: 0.3278
Epoch 8/20

Epoch 61/2000
720/720 [==============================] - 0s 11us/step - loss: 0.9214 - acc: 0.5792 - val_loss: 3.0362 - val_acc: 0.4333
Epoch 62/2000
720/720 [==============================] - 0s 11us/step - loss: 0.9354 - acc: 0.6000 - val_loss: 3.0033 - val_acc: 0.4333
Epoch 63/2000
720/720 [==============================] - 0s 11us/step - loss: 0.9210 - acc: 0.6028 - val_loss: 2.9736 - val_acc: 0.4389
Epoch 64/2000
720/720 [==============================] - 0s 11us/step - loss: 0.8508 - acc: 0.6319 - val_loss: 2.9451 - val_acc: 0.4500
Epoch 65/2000
720/720 [==============================] - 0s 13us/step - loss: 0.8746 - acc: 0.6333 - val_loss: 2.9127 - val_acc: 0.4556
Epoch 66/2000
720/720 [==============================] - 0s 12us/step - loss: 0.9207 - acc: 0.6194 - val_loss: 2.8766 - val_acc: 0.4611
Epoch 67/2000
720/720 [==============================] - 0s 11us/step - loss: 0.8981 - acc: 0.6083 - val_loss: 2.8457 - val_acc: 0.4722
Epoch 68/2000
720/720 [=========================

Epoch 122/2000
720/720 [==============================] - 0s 21us/step - loss: 0.8078 - acc: 0.6611 - val_loss: 1.5626 - val_acc: 0.4944
Epoch 123/2000
720/720 [==============================] - 0s 14us/step - loss: 0.8178 - acc: 0.6792 - val_loss: 1.5455 - val_acc: 0.5000
Epoch 124/2000
720/720 [==============================] - 0s 14us/step - loss: 0.7782 - acc: 0.6722 - val_loss: 1.5276 - val_acc: 0.5000
Epoch 125/2000
720/720 [==============================] - 0s 14us/step - loss: 0.8113 - acc: 0.6444 - val_loss: 1.5099 - val_acc: 0.5000
Epoch 126/2000
720/720 [==============================] - 0s 14us/step - loss: 0.7920 - acc: 0.6750 - val_loss: 1.4951 - val_acc: 0.5056
Epoch 127/2000
720/720 [==============================] - 0s 26us/step - loss: 0.7816 - acc: 0.6514 - val_loss: 1.4808 - val_acc: 0.5056
Epoch 128/2000
720/720 [==============================] - 0s 26us/step - loss: 0.7942 - acc: 0.6736 - val_loss: 1.4665 - val_acc: 0.5056
Epoch 129/2000
720/720 [=================

Epoch 182/2000
720/720 [==============================] - 0s 11us/step - loss: 0.7719 - acc: 0.6986 - val_loss: 0.9417 - val_acc: 0.6056
Epoch 183/2000
720/720 [==============================] - 0s 12us/step - loss: 0.7558 - acc: 0.6861 - val_loss: 0.9357 - val_acc: 0.6111
Epoch 184/2000
720/720 [==============================] - 0s 14us/step - loss: 0.7671 - acc: 0.6889 - val_loss: 0.9291 - val_acc: 0.6222
Epoch 185/2000
720/720 [==============================] - 0s 5us/step - loss: 0.8097 - acc: 0.6542 - val_loss: 0.9225 - val_acc: 0.6222
Epoch 186/2000
720/720 [==============================] - 0s 28us/step - loss: 0.7636 - acc: 0.6750 - val_loss: 0.9165 - val_acc: 0.6222
Epoch 187/2000
720/720 [==============================] - 0s 24us/step - loss: 0.7863 - acc: 0.6750 - val_loss: 0.9106 - val_acc: 0.6278
Epoch 188/2000
720/720 [==============================] - 0s 17us/step - loss: 0.7571 - acc: 0.7056 - val_loss: 0.9051 - val_acc: 0.6333
Epoch 189/2000
720/720 [==================

Epoch 242/2000
720/720 [==============================] - 0s 25us/step - loss: 0.7023 - acc: 0.7319 - val_loss: 0.6869 - val_acc: 0.7056
Epoch 243/2000
720/720 [==============================] - 0s 10us/step - loss: 0.7370 - acc: 0.7056 - val_loss: 0.6861 - val_acc: 0.7056
Epoch 244/2000
720/720 [==============================] - 0s 0us/step - loss: 0.7512 - acc: 0.6931 - val_loss: 0.6848 - val_acc: 0.7056
Epoch 245/2000
720/720 [==============================] - 0s 22us/step - loss: 0.7323 - acc: 0.6986 - val_loss: 0.6838 - val_acc: 0.7056
Epoch 246/2000
720/720 [==============================] - 0s 11us/step - loss: 0.7476 - acc: 0.7028 - val_loss: 0.6819 - val_acc: 0.7000
Epoch 247/2000
720/720 [==============================] - 0s 22us/step - loss: 0.7012 - acc: 0.7167 - val_loss: 0.6795 - val_acc: 0.7000
Epoch 248/2000
720/720 [==============================] - 0s 0us/step - loss: 0.7518 - acc: 0.6806 - val_loss: 0.6767 - val_acc: 0.7056
Epoch 249/2000
720/720 [===================

...

Epoch 1745/2000
720/720 [==============================] - 0s 8us/step - loss: 0.5394 - acc: 0.7986 - val_loss: 0.5370 - val_acc: 0.7944
Epoch 1746/2000
720/720 [==============================] - 0s 12us/step - loss: 0.5631 - acc: 0.7917 - val_loss: 0.5359 - val_acc: 0.7944
Epoch 1747/2000
720/720 [==============================] - 0s 8us/step - loss: 0.5665 - acc: 0.7764 - val_loss: 0.5349 - val_acc: 0.8000
Epoch 1748/2000
720/720 [==============================] - 0s 8us/step - loss: 0.5634 - acc: 0.7847 - val_loss: 0.5341 - val_acc: 0.8056
Epoch 1749/2000
720/720 [==============================] - 0s 14us/step - loss: 0.5606 - acc: 0.7847 - val_loss: 0.5334 - val_acc: 0.8000
Epoch 1750/2000
720/720 [==============================] - 0s 14us/step - loss: 0.5460 - acc: 0.8000 - val_loss: 0.5328 - val_acc: 0.8000
Epoch 1751/2000
720/720 [==============================] - 0s 11us/step - loss: 0.5713 - acc: 0.7847 - val_loss: 0.5320 - val_acc: 0.8000
Epoch 1752/2000
720/720 [============

Epoch 1805/2000
720/720 [==============================] - 0s 15us/step - loss: 0.5437 - acc: 0.8000 - val_loss: 0.5395 - val_acc: 0.7778
Epoch 1806/2000
720/720 [==============================] - 0s 22us/step - loss: 0.5723 - acc: 0.7764 - val_loss: 0.5401 - val_acc: 0.7778
Epoch 1807/2000
720/720 [==============================] - 0s 12us/step - loss: 0.5517 - acc: 0.7861 - val_loss: 0.5411 - val_acc: 0.7778
Epoch 1808/2000
720/720 [==============================] - 0s 14us/step - loss: 0.5647 - acc: 0.7875 - val_loss: 0.5419 - val_acc: 0.7778
Epoch 1809/2000
720/720 [==============================] - 0s 14us/step - loss: 0.5426 - acc: 0.7861 - val_loss: 0.5424 - val_acc: 0.7778
Epoch 1810/2000
720/720 [==============================] - 0s 15us/step - loss: 0.5477 - acc: 0.7931 - val_loss: 0.5426 - val_acc: 0.7722
Epoch 1811/2000
720/720 [==============================] - 0s 14us/step - loss: 0.5917 - acc: 0.7694 - val_loss: 0.5435 - val_acc: 0.7722
Epoch 1812/2000
720/720 [=========

Epoch 1865/2000
720/720 [==============================] - 0s 12us/step - loss: 0.5393 - acc: 0.8028 - val_loss: 0.5351 - val_acc: 0.7778
Epoch 1866/2000
720/720 [==============================] - 0s 13us/step - loss: 0.5564 - acc: 0.7806 - val_loss: 0.5358 - val_acc: 0.7778
Epoch 1867/2000
720/720 [==============================] - 0s 10us/step - loss: 0.5501 - acc: 0.7903 - val_loss: 0.5363 - val_acc: 0.7778
Epoch 1868/2000
720/720 [==============================] - 0s 14us/step - loss: 0.5749 - acc: 0.7861 - val_loss: 0.5364 - val_acc: 0.7722
Epoch 1869/2000
720/720 [==============================] - 0s 14us/step - loss: 0.5441 - acc: 0.7847 - val_loss: 0.5365 - val_acc: 0.7778
Epoch 1870/2000
720/720 [==============================] - 0s 11us/step - loss: 0.5676 - acc: 0.7889 - val_loss: 0.5361 - val_acc: 0.7778
Epoch 1871/2000
720/720 [==============================] - 0s 16us/step - loss: 0.5394 - acc: 0.8000 - val_loss: 0.5358 - val_acc: 0.7778
Epoch 1872/2000
720/720 [=========

Epoch 1925/2000
720/720 [==============================] - 0s 14us/step - loss: 0.5465 - acc: 0.8125 - val_loss: 0.5322 - val_acc: 0.8222
Epoch 1926/2000
720/720 [==============================] - 0s 13us/step - loss: 0.5477 - acc: 0.7847 - val_loss: 0.5329 - val_acc: 0.8167
Epoch 1927/2000
720/720 [==============================] - 0s 15us/step - loss: 0.5627 - acc: 0.7833 - val_loss: 0.5325 - val_acc: 0.8167
Epoch 1928/2000
720/720 [==============================] - 0s 14us/step - loss: 0.5398 - acc: 0.7833 - val_loss: 0.5323 - val_acc: 0.8111
Epoch 1929/2000
720/720 [==============================] - 0s 12us/step - loss: 0.5418 - acc: 0.7778 - val_loss: 0.5320 - val_acc: 0.8111
Epoch 1930/2000
720/720 [==============================] - 0s 3us/step - loss: 0.5473 - acc: 0.7917 - val_loss: 0.5314 - val_acc: 0.8167
Epoch 1931/2000
720/720 [==============================] - 0s 6us/step - loss: 0.5358 - acc: 0.8014 - val_loss: 0.5306 - val_acc: 0.8167
Epoch 1932/2000
720/720 [===========

Epoch 1985/2000
720/720 [==============================] - 0s 14us/step - loss: 0.5776 - acc: 0.7917 - val_loss: 0.5236 - val_acc: 0.8000
Epoch 1986/2000
720/720 [==============================] - 0s 26us/step - loss: 0.5668 - acc: 0.7875 - val_loss: 0.5222 - val_acc: 0.8000
Epoch 1987/2000
720/720 [==============================] - 0s 14us/step - loss: 0.5274 - acc: 0.7875 - val_loss: 0.5212 - val_acc: 0.8000
Epoch 1988/2000
720/720 [==============================] - 0s 15us/step - loss: 0.5272 - acc: 0.7972 - val_loss: 0.5204 - val_acc: 0.7944
Epoch 1989/2000
720/720 [==============================] - 0s 12us/step - loss: 0.5571 - acc: 0.7778 - val_loss: 0.5207 - val_acc: 0.7944
Epoch 1990/2000
720/720 [==============================] - 0s 12us/step - loss: 0.5577 - acc: 0.7792 - val_loss: 0.5215 - val_acc: 0.7944
Epoch 1991/2000
720/720 [==============================] - 0s 12us/step - loss: 0.5647 - acc: 0.7972 - val_loss: 0.5225 - val_acc: 0.7889
Epoch 1992/2000
720/720 [=========

In [22]:
train_loss, train_accuracy = model.evaluate(X_train, y_train, batch_size=BATCH_SIZE)
train_loss, train_accuracy

900/900 [==============================] - 0s 34us/step


(0.4888240694999695, 0.8088889122009277)

In [23]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE)
test_loss, test_accuracy

600/600 [==============================] - 0s 5us/step


(0.5224040746688843, 0.7933333516120911)

In [25]:
model.save('insurance.hdf5')

In [24]:
!ls -l

total 6925
-rw-r--r-- 1 olive 197609    9293 Aug  8 10:59 0-generate.ipynb
drwxr-xr-x 1 olive 197609       0 Aug  8 10:58 data
drwxr-xr-x 1 olive 197609       0 Aug  8 12:01 figures
-rw-r--r-- 1 olive 197609  179808 Aug 13 18:27 insurance.hdf5
-rw-r--r-- 1 olive 197609   26822 Aug 13 19:04 insurance-customers-1500.csv
-rw-r--r-- 1 olive 197609  179704 Aug 13 19:04 insurance-simple.hdf5
drwxr-xr-x 1 olive 197609       0 Aug 13 18:59 model
-rw-r--r-- 1 olive 197609      88 Aug  9 17:14 sample_insurance.json
drwxr-xr-x 1 olive 197609       0 Aug  9 16:15 tf
drwxr-xr-x 1 olive 197609       0 Aug 13 19:02 tfjs
-rw-r--r-- 1 olive 197609  634707 Aug  9 13:01 U3-M1-example.ipynb
-rw-r--r-- 1 olive 197609 3620457 Aug  9 10:31 U3-M2-nn-intro.ipynb
-rw-r--r-- 1 olive 197609 2002487 Aug  9 12:14 U3-M3-nn-no-bullshit.ipynb
-rw-r--r-- 1 olive 197609  338175 Aug 13 19:06 U3-M3-nn-simplified.ipynb
-rw-r--r-- 1 olive 197609    8055 Aug  9 17:30 U4-M1-Preparing TensorFlow models.ipynb
-rw-r--r-- 1 olive